# Form13
This notebook described how to use Neo4j and SageMaker together.  In it you connect to a Neo4j instance, load data and compute an embedding.  You then load that data into AWS S3 for later consumptin by SageMaker.

You're going to want to run it in Colab or another environment that allows outbound access on port 7687.  Note that AWS SageMaker Studio blocks outbound access on all ports except for 53, 80 and 443.  You can click the link below to open the notebook in Colab.

<p>
<a href="https://colab.research.google.com/github/neo4j-partners/neo4j-sagemaker/blob/main/form13/embedding.ipynb" target="_blank">
<img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
</a>
</p>

## Deploy Neo4j
You're going to need a Neo4j deployment to run this lab.  The easiest way to get that is via the [AWS Marketplace](https://aws.amazon.com/marketplace/seller-profile?id=23ec694a-d2af-4641-b4d3-b7201ab2f5f9).  Select "Neo4j Enterprise Edition" and deploy that.  Suggested parameters are:

* Graph Database Version - 4.4.6
* Install Graph Data Sceince - True
* Graph Data Science License Key - None
* Install Bloom - True
* Bloom License Key - None
* Password - Enter something here
* Node Count - 1
* Instance Type - t3.xlarge
* Key Name - Pick a key you have
* SSH CIDR - 0.0.0.0/0

The Marketplace listing deploys an ASG.  When deployment is complete, you can get the IP address of your Neo4j node from that ASG.  You can view deployed ASGs [here](https://us-east-1.console.aws.amazon.com/ec2autoscaling).

## Using the Neo4j API
Now that we have a Neo4j deployment, let's connect to Neo4j.  First off, install the Neo4j Graph Data Science package.

In [ ]:
%pip install graphdatascience

Now, you're going to need the connection string and credentials from the deployment you created above.

In [ ]:
# Edit these variables!
DB_URL = 'neo4j://ec2-3-91-14-235.compute-1.amazonaws.com:7687'
DB_PASS = 'foo123'

# You can leave this default
DB_USER = 'neo4j'

In [ ]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(DB_URL, auth=(DB_USER, DB_PASS))

# Load Data into Neo4j
Now that we've got our connection object, let's load the dataset into Neo4j.

The dataset is pulled from the SEC's EDGAR database. These are public filings of something called Form 13. Asset managers with over $100m AUM are required to submit Form 13 quarterly. That's then made available to the public over http. The csvs linked above were pulled from EDGAR using some python scripts. If you're curious, they're all available [here](https://github.com/neo4j-partners/neo4j-sec-edgar-form13). We've filtered the data to only include filings over $10m in value.

We're going to create constratints for our data.


In [ ]:
result = gds.run_cypher('CREATE CONSTRAINT IF NOT EXISTS ON (p:Company) ASSERT (p.cusip) IS NODE KEY;')
display(result)

result = gds.run_cypher('CREATE CONSTRAINT IF NOT EXISTS ON (p:Manager) ASSERT (p.filingManager) IS NODE KEY;')
display(result)

result = gds.run_cypher('CREATE CONSTRAINT IF NOT EXISTS ON (p:Holding) ASSERT (p.filingManager, p.cusip, p.reportCalendarOrQuarter) IS NODE KEY;')
display(result)

Now let's load the nodes.

In [ ]:
result = gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/neo4j-datasets/form13/2021.csv' AS row
        MERGE (c:Company {cusip:row.cusip})
        ON CREATE SET
            c.nameOfIssuer=row.nameOfIssuer
    """
)
display(result)

In [ ]:
result = gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/neo4j-datasets/form13/2021.csv' AS row
        MERGE (m:Manager {filingManager:row.filingManager})
    """
)
display(result)

In [ ]:
result = gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/neo4j-datasets/form13/2021.csv' AS row
        MERGE (h:Holding {filingManager:row.filingManager, cusip:row.cusip, reportCalendarOrQuarter:row.reportCalendarOrQuarter})
        ON CREATE SET
            h.value=row.value, 
            h.shares=row.shares,
            h.target=row.target,
            h.nameOfIssuer=row.nameOfIssuer
    """
)
display(result)

Now let's create relationships between those nodes.

In [ ]:
result = gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/neo4j-datasets/form13/2021.csv' AS row
        MATCH (m:Manager {filingManager:row.filingManager})
        MATCH (h:Holding {filingManager:row.filingManager, cusip:row.cusip, reportCalendarOrQuarter:row.reportCalendarOrQuarter})
        MERGE (m)-[r:OWNS]->(h)
    """
)
display(result)

In [ ]:
result = gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM 'https://storage.googleapis.com/neo4j-datasets/form13/2021.csv' AS row
        MATCH (h:Holding {filingManager:row.filingManager, cusip:row.cusip, reportCalendarOrQuarter:row.reportCalendarOrQuarter})
        MATCH (c:Company {cusip:row.cusip})
        MERGE (h)-[r:PARTOF]->(c)
    """
)
display(result)

# Graph Data Science
Now we're going to use Neo4j Graph Data Science to create an in memory graph represtation of the data.  We'll enhance that represation with features we engineer using a graph embedding.

In [ ]:
result = gds.run_cypher(
  """
    CALL gds.graph.project(
      'mygraph',
      ['Company', 'Manager', 'Holding'],
      {
          OWNS: {orientation: 'UNDIRECTED'},
          PARTOF: {orientation: 'UNDIRECTED'}
      }
    )
    YIELD
      graphName AS graph,
      relationshipProjection AS readProjection,
      nodeCount AS nodes,
      relationshipCount AS rels
  """
)
display(result)

Note, if you get an error saying the graph already exists, that's probably because you ran this code before. You can destroy it using this command:

In [ ]:
result = gds.run_cypher(
  """
    CALL gds.graph.drop('mygraph')
  """
)
display(result)

Now, let's list the details of the graph to make sure the projection was created as we want.

In [ ]:
result = gds.run_cypher(
  """
    CALL gds.graph.list()
  """
)
display(result)

Now we can generate an embedding from that graph. This is a new feature we can use in our predictions. We're using FastRP, which is a more full featured and higher performance of Node2Vec. You can learn more about that [here](https://neo4j.com/docs/graph-data-science/current/algorithms/fastrp/).

There are a bunch of parameters we could adjust in this.  One of the most obvious is the embeddingDimension.  The documentation covers many more.

In [ ]:
result = gds.run_cypher(
  """
  CALL gds.fastRP.mutate('mygraph',{
    embeddingDimension: 16,
    randomSeed: 1,
    mutateProperty:'embedding'
  })
  """
)
display(result)

That creates an embedding for each node type.  However, we only want the embedding on the nodes of type holding.

We're going to take the embedding from our projection and write it to the holding nodes in the underlying database.

In [ ]:
result = gds.run_cypher(
  """
    CALL gds.graph.writeNodeProperties('mygraph', ['embedding'], ['Holding'])
    YIELD writeMillis
  """
)
display(result)

In [ ]:
result = gds.run_cypher(
  """
    MATCH (n:Holding) RETURN n
  """
)
display(result)

Note that this query will take 2-3 minutes to run as it's grabbing nearly half a million nodes along with all their properties and our new embedding.

In [ ]:
import pandas as pd
df = pd.DataFrame([dict(record.items()) for record in result['n']])
df

Note that the embedding row is an array. To make this dataset more consumable, we should flatten that out into multiple individual features: embedding_0, embedding_1, ... embedding_n.


In [ ]:
embeddings = pd.DataFrame(df['embedding'].values.tolist()).add_prefix("embedding_")
merged = df.drop(columns=['embedding']).merge(embeddings, left_index=True, right_index=True)
merged

Now that we have the data formatted properly, let's split it into a training and a testing set and write those to disk.

In [ ]:
df = merged

df['split']=df['reportCalendarOrQuarter']
df['split']=df['split'].replace(['03-31-2021', '06-30-2021', '09-30-2021'], ['TRAIN', 'VALIDATE', 'TEST'])

df = df.drop(columns=['reportCalendarOrQuarter'])

df.to_csv('embedding.csv', index=False)

# Upload to Amazom S3
Now let's create a bucket and upload our data set to it.  Then we'll be able to access the data in SageMaker in the next lab.

In [10]:
filename='embedding.csv'

#to do

In [ ]:
import boto
import boto.s3
import sys
from boto.s3.key import Key

AWS_ACCESS_KEY_ID = ''
AWS_SECRET_ACCESS_KEY = ''

bucket_name = AWS_ACCESS_KEY_ID.lower() + '-dump'
conn = boto.connect_s3(AWS_ACCESS_KEY_ID,
        AWS_SECRET_ACCESS_KEY)


bucket = conn.create_bucket(bucket_name,
    location=boto.s3.connection.Location.DEFAULT)

testfile = "replace this with an actual filename"
print 'Uploading %s to Amazon S3 bucket %s' % \
   (testfile, bucket_name)

def percent_cb(complete, total):
    sys.stdout.write('.')
    sys.stdout.flush()


k = Key(bucket)
k.key = 'my test file'
k.set_contents_from_filename(testfile,
    cb=percent_cb, num_cb=10)